<a href="https://colab.research.google.com/github/carricarte/neuromatch/blob/main/meta_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT RELEVANT LIBRARIES**

In [10]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import re
from datasets import load_dataset
from transformers import BertTokenizer

**HELPER FUNCTION (FROM TEMPLATE)**

In [11]:

_patterns = [r"\'", r"\"", r"\.", r"<br \/>", r",", r"\(", r"\)", r"\!", r"\?", r"\;", r"\:", r"\s+"]

_replacements = [" '  ", "", " . ", " ", " , ", " ( ", " ) ", " ! ", " ? ", " ", " ", " "]

_patterns_dict = list((re.compile(p), r) for p, r in zip(_patterns, _replacements))

def _basic_english_normalize(line):
    r"""
    Basic normalization for a line of text.
    Normalization includes
    - lowercasing
    - complete some basic text normalization for English words as follows:
        add spaces before and after '\''
        remove '\"',
        add spaces before and after '.'
        replace '<br \/>'with single space
        add spaces before and after ','
        add spaces before and after '('
        add spaces before and after ')'
        add spaces before and after '!'
        add spaces before and after '?'
        replace ';' with single space
        replace ':' with single space
        replace multiple spaces with single space

    Returns a list of tokens after splitting on whitespace.
    """

    line = line.lower()
    for pattern_re, replaced_str in _patterns_dict:
        line = pattern_re.sub(replaced_str, line)
    return line.split()

In [12]:
## Here we should read the data from a CSV file or any other source.
# then visualize it using pandas ot matplot to gain insights from it.

In [13]:
# Data preprocessing
def preprocess_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())
    # Remove punctuation and non-alphabetic characters
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if len(word) > 1]  # Remove single-character tokens
    tokens= [word for word in tokens if re.search(r'[a-zA-Z]', word)]  # Keep only words with alphabetic characters
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

**VISUALIZE DATA**

In [14]:
## Visulaizing
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud
# # Set the style for seaborn
# sns.set(style='whitegrid')
# # Set the color palette for seaborn
# sns.set_palette('pastel')

# # --- Visualization 1: Label Distribution ---
# plt.figure(figsize=(6, 4))
# df['label'].value_counts().sort_index().plot(kind='bar', color=['red', 'gray', 'green'])
# plt.title('Sentiment Label Distribution')
# plt.xlabel('Sentiment')
# plt.ylabel('Number of Samples')
# plt.xticks(ticks=[0, 1, 2], labels=['Negative (-1)', 'Neutral (0)', 'Positive (1)'], rotation=0)
# plt.grid(axis='y')
# plt.tight_layout()
# plt.show()

# # --- Visualization 2: Word Clouds per Sentiment ---
# fig, axes = plt.subplots(1, 3, figsize=(18, 6))
# sentiments = {-1: 'Negative', 0: 'Neutral', 1: 'Positive'}
# colors = {-1: 'Reds', 0: 'Greys', 1: 'Greens'}

# for idx, sentiment in enumerate(sentiments.keys()):
#     texts = ' '.join(df[df['label'] == sentiment]['text'].astype(str))
#     if texts.strip():  # avoid errors for empty sentiment class
#         wordcloud = WordCloud(width=600, height=400, background_color='white',
#                               colormap=colors[sentiment], max_words=100).generate(texts)
#         axes[idx].imshow(wordcloud, interpolation='bilinear')
#         axes[idx].set_title(f'{sentiments[sentiment]} Sentiment')
#         axes[idx].axis('off')
#     else:
#         axes[idx].set_title(f'No Data for {sentiments[sentiment]}')
#         axes[idx].axis('off')

# plt.tight_layout()
# plt.show()

**IMPORT DATA**

In [15]:
dataset = load_dataset('sentiment140')
df = dataset["train"].to_pandas()

# df = pd.read_csv('PATH/TO/CSV/FILE', encoding='ISO-8859-1')
df = df.rename(columns={'0': 'polarity', '1467810369': 'user', 'Mon Apr 06 22:19:45 PDT 2009': 'date', 'NO_QUERY': 'query'
                        , '_TheSpecialOne_': 'user', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": 'text' })
df = df[['polarity', 'user', 'date', 'query', 'user', 'text']]
df.head()

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

**ENCODING**

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# Tokenize the text data
def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# tokenize the preprocessed text data
encoded = tokenize_texts(X_orig[:, 0].tolist())  # Use the first column for text data
# Convert labels to tensor
labels = torch.tensor(X_orig[:, 1].astype(int))  # Use the second column
# Create a dataset class